In [1]:
import utils
inp = utils.get_input(2019, 17)[:-1]
print(inp[:96])

Found cached input for 2019 day 17
1,330,331,332,109,3448,1101,0,1182,16,1101,0,1439,24,101,0,0,570,1006,570,36,102,1,571,0,1001,57


## machine

In [2]:
import operator
from functools import partial
from itertools import repeat, chain
from collections import namedtuple, defaultdict


class IntCode:
    Op = namedtuple('Op', ['func', 'params', 'resolve_last_as_ptr'])

    def __init__(self, program, pointer=0, rel_base=0, inputs=None, **kwargs):
        self._mem = defaultdict(int, enumerate(program))
        self.ptr = pointer
        self.rel = rel_base
        self.state = "started"
        self.inputs = inputs or []
        self.output = []
        self._kwargs = kwargs
        self._ops = {
            1: IntCode.Op(partial(IntCode._math, operator.add), 3, True),
            2: IntCode.Op(partial(IntCode._math, operator.mul), 3, True),
            3: IntCode.Op(IntCode._inp, 1, True),
            4: IntCode.Op(IntCode._out, 1, False),
            5: IntCode.Op(partial(IntCode._jump, lambda x: x != 0), 2, False),
            6: IntCode.Op(partial(IntCode._jump, lambda x: x == 0), 2, False),
            7: IntCode.Op(partial(IntCode._math, operator.lt), 3, True),
            8: IntCode.Op(partial(IntCode._math, operator.eq), 3, True),
            9: IntCode.Op(IntCode._base, 1, False),
            99: IntCode.Op(IntCode._halt, 0, False),
        }

    @property
    def mem(self):
        return list(self._mem.values())

    def run(self, inputs=None):
        self.inputs.extend(inputs or [])

        while self.state != "halted":
            instruction = self._mem[self.ptr]
            opcode, modes = self._opcode(instruction), self._modes(instruction)
            op = self._ops[opcode]
            params_raw = [self._mem[i] for i in range(self.ptr + 1, self.ptr + op.params + 1)]
            params = self.resolve(params_raw, modes, op.resolve_last_as_ptr)

            orig_ptr = self.ptr
            self._mem, self.ptr, self.rel, output, self.state = \
                op.func(self._mem, self.ptr, self.rel, *params, inputs=self.inputs, **self._kwargs)
            if output is not None:
                self.output.append(output)

            if self._kwargs.get('_debug', False):
                print(f"{instruction},{','.join(map(str, params_raw))}",
                      f"-> {opcode:2d} {modes} {params}",
                      f"-> {output, self.ptr, self.rel}")

            if (output is not None) or (self.state == "blocked"):
                yield self

            if self.ptr == orig_ptr:
                self.ptr = self.ptr + op.params + 1
        yield self

    def resolve(self, params, modes, resolve_last_as_ptr):
        def _resolve(p, m, as_ptr):
            if int(m) == 0:
                return self._mem[p] if not as_ptr else p
            elif int(m) == 2:
                return self._mem[self.rel + p] if not as_ptr else self.rel + p
            return p
        resolve_as_ptr = chain(repeat(False, len(params) - 1), [resolve_last_as_ptr])
        return list(map(lambda t: _resolve(*t), zip(params, modes, resolve_as_ptr)))

    @staticmethod
    def _opcode(instruction):
        return int(str(instruction)[-2:])

    @staticmethod
    def _modes(instruction):
        return f"{instruction:05d}"[:3][::-1]
    
    # operations
    
    @staticmethod
    def _math(func, mem, ptr, rel, a, b, out, **kwargs):
        mem[out] = int(func(a, b))
        return mem, ptr, rel, None, "running"

    @staticmethod
    def _inp(mem, ptr, rel, out, inputs, **kwargs):
        if len(inputs):
            mem[out] = int(inputs.pop(0))
            return mem, ptr, rel, None, "running"
        return mem, ptr, rel, None, "blocked"

    @staticmethod
    def _out(mem, ptr, rel, val, **kwargs):
        return mem, ptr, rel, val, "running"

    @staticmethod
    def _jump(func, mem, ptr, rel, cond, val, **kwargs):
        return mem, (val if func(cond) else ptr), rel, None, "running"

    @staticmethod
    def _base(mem, ptr, rel, val, **kwargs):
        return mem, ptr, (rel + val), None, "running"
    
    @staticmethod
    def _halt(mem, ptr, rel, *args, **kwargs):
        return mem, ptr, rel, None, "halted"


In [3]:
from itertools import groupby


program = map(int, inp.split(','))
vm = IntCode(program)
runner = vm.run()

while vm.state != "halted":
    vm = next(runner)

out = vm.output
out = list(map(chr, out[:-1]))
# print(''.join(out))

# (y, x)
rows = [list(g) for k, g in groupby(out, lambda x: x == '\n') if not k]
intersections = []

for y, row in enumerate(rows):
    for x, v in enumerate(row):
        if v == '#' and (0 < x < len(row) - 1) and (0 < y < len(rows) - 1):
            cross = [(1, 0), (-1, 0), (0, 1), (0, -1)]
            if all(rows[y + dy][x + dx] == '#' for dx, dy in cross):
                intersections.append((x, y))

print(sum(a * b for a, b in intersections))

6052


In [4]:
import re
from functools import partial
from itertools import groupby, combinations
from collections import Counter
from IPython.display import clear_output


ORIENT = [(0, -1), (1, 0), (0, 1), (-1, 0)]
ORIENT_SYMBOL = ['^', '>', 'v', '<']


def get_image(inp):
    program = map(int, inp.split(','))
    vm = IntCode(program)
    runner = vm.run()

    while vm.state != "halted":
        vm = next(runner)

    out = vm.output
    return list(map(chr, out[:-1]))


def matrix(rows):
    m = {}
    for y, row in enumerate(rows):
        for x, v in enumerate(row):
            m[(x, y)] = v
    return m, len(row), len(rows)


def invert(d):
    d_ = defaultdict(list)
    for k, v in d.items():
        d_[v].append(k)
    return d_


def robot(m):
    for x, o in zip(ORIENT_SYMBOL, ORIENT):
        if x in invert(m):
            return (invert(m)[x][0], o)
    return None


def _direction(max_x, max_y, m, loc):
    def _axis(coord):
        return 'x' if coord[0] != 0 else 'y'

    for dx, dy in ORIENT:
        x, y = loc[0][0] + dx, loc[0][1] + dy
        if (0 <= x < max_x) and (0 <= y < max_y):
            if m[(x, y)] == '#' and _axis(loc[1]) != _axis((dx, dy)):
                return (dx, dy)
    return None


def turn(prev, curr):
    i = ORIENT.index(prev)
    if ORIENT[(i + 1) % len(ORIENT)] == curr:
        return 'R'
    return 'L'


def trace(m, loc):
    def _step(vec):
        return ((vec[0][0] + vec[1][0], vec[0][1] + vec[1][1]), vec[1])
    curr = loc
    while m.get(_step(curr)[0], None) == '#':
        curr = _step(curr)
    return curr


def distance(prev, curr):
    return max(abs(curr[0] - prev[0]), abs(curr[1] - prev[1]))


def path(rows):
    m, max_x, max_y = matrix(rows)
    direction = partial(_direction, max_x, max_y, m)

    prev = robot(m)
    path = []
    dir_ = direction(prev)

    while dir_ is not None:
        curr = (prev[0], dir_)
        path.append(turn(prev[1], curr[1]))
        prev = trace(m, curr)
        path.append(distance(curr[0], prev[0]))
        dir_ = direction(prev)
    return path


def subseqs(s):

    def replace(s, old, new):
        return s.replace(old, new)

    def rreplace(s, old, new):
        li = s.rsplit(old)
        return new.join(li)

    subs = [s[i:j + 2] for i in range(0, len(s), 2) for j in range(i, len(s), 2)]
    subs = list(map(lambda sub: ''.join(map(str, sub)), subs))
    repeated = [k for k, v in Counter(subs).items() if v >= 2 and 2 < len(k) <= 10]

    s = ''.join(map(str, s))
    for a, b, c in combinations(repeated, 3):
        funcs = [replace, rreplace]
        for fa in funcs:
            for fb in funcs:
                for fc in funcs:
                    if not fc(fb(fa(s, a, ''), b, ''), c, ''):
                        yield fc(fb(fa(s, a, 'A'), b, 'B'), c, 'C'), a, b, c


def encode(s):
    tokens = re.split(r'(\d+)', s)[:-1] if re.search(r'\d', s) else list(s)
    return list(map(ord, ','.join(tokens))) + [ord('\n')]


img = get_image(inp)
rows = [list(g) for k, g in groupby(img, lambda x: x == '\n') if not k]
p = path(rows)
main, a, b, c = sorted(list(subseqs(p)), key=lambda t: len(t[0]))[0]

video = True
inputs = list(chain(*map(encode, [main, a, b, c, 'y' if video else 'n'])))
program = list(map(int, inp.split(',')))
program[0] = 2

In [5]:
vm = IntCode(program, inputs=list(inputs))
runner = vm.run()

streaming = False
rendered = False

while vm.state != "halted":
    out = ''.join(list(map(chr, vm.output)))

    if '\n\n' in out:
        rendered = True

    if streaming and rendered:
        if video:
            clear_output(wait=True)
            print(out)
        rendered = False
        vm.output = []

    if 'Continuous video feed?\n' in out:
        streaming = True
        vm.output = []

    vm = next(runner)

print(vm.output[-1])

#######..................................
#.....#..................................
#.....#..................................
#.....#..................................
#.....#..................................
#.....#..................................
#.....#.......................#####......
#.....#.......................#...#......
#.....#.......................#...#......
#.....#.......................#...#......
###########.................###########..
......#...#.................#.#...#...#..
......#######...........#######...#...#..
..........#.#...........#...#.....#...#..
..........#.#...........#...#.....#...#..
..........#.#...........#...#.....#...#..
..........#######.......#.#############..
............#...#.......#.#.#.....#......
............#############.#.#.....#######
................#.........#.#............
................#.....#######............
................#.....#...#..............
................#...#######..............
................#...#.#...........